In [ ]:

%tensorflow_version 1.x
import numpy as np
from numpy.random import seed
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.framework import ops

!rm -f *.npy


!wget https://qin.ee/bearings/e90769db282a3b585b97cbb05183423d/XsynallDEenv.npy
!wget https://qin.ee/bearings/e90769db282a3b585b97cbb05183423d/XreallDEenv.npy
!wget https://qin.ee/bearings/e90769db282a3b585b97cbb05183423d/ysynallDEenv.npy
!wget https://qin.ee/bearings/e90769db282a3b585b97cbb05183423d/yreallDEenv.npy

!md5sum XsynallDEenv.npy

# 4096 100 -> (4000, 1000)



TensorFlow 1.x selected.
--2021-01-20 16:21:00--  https://qin.ee/bearings/e90769db282a3b585b97cbb05183423d/XsynallDEenv.npy
Resolving qin.ee (qin.ee)... 35.177.27.26
Connecting to qin.ee (qin.ee)|35.177.27.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32000128 (31M) [application/octet-stream]
Saving to: ‘XsynallDEenv.npy’

XsynallDEenv.npy    100%[===================>]  30.52M  95.6MB/s    in 0.3s    

2021-01-20 16:21:02 (95.6 MB/s) - ‘XsynallDEenv.npy’ saved [32000128/32000128]

--2021-01-20 16:21:02--  https://qin.ee/bearings/e90769db282a3b585b97cbb05183423d/XreallDEenv.npy
Resolving qin.ee (qin.ee)... 35.177.27.26
Connecting to qin.ee (qin.ee)|35.177.27.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32000128 (31M) [application/octet-stream]
Saving to: ‘XreallDEenv.npy’

XreallDEenv.npy     100%[===================>]  30.52M   103MB/s    in 0.3s    

2021-01-20 16:21:13 (103 MB/s) - ‘XreallDEenv.npy’ saved [32000128/320001

In [ ]:
class FlipGradientBuilder(object):
    def __init__(self):
        self.num_calls = 0

    def __call__(self, x, l=1.0):
        grad_name = "FlipGradient%d" % self.num_calls
        @ops.RegisterGradient(grad_name)
        def _flip_gradients(op, grad):
            return [tf.negative(grad) * l]
        
        g = tf.get_default_graph()
        with g.gradient_override_map({"Identity": grad_name}):
            y = tf.identity(x)
            
        self.num_calls += 1
        return y
    
flip_gradient = FlipGradientBuilder()

In [ ]:
def eval(pred, y):
    y0 = pred[y==0]
    y1 = pred[y==1]
    y2 = pred[y==2]
    y3 = pred[y==3]
    acc = np.sum(y0==0)/len(y0)
    acc += np.sum(y1==1)/len(y1)
    acc += np.sum(y2==2)/len(y2)
    acc += np.sum(y3==3)/len(y3)
    print("per class", np.sum(y0==0)/len(y0), np.sum(y1==1)/len(y1), np.sum(y2==2)/len(y2), np.sum(y3==3)/len(y3))
    return acc/4



def feature_extractor(x, training=True, reuse=False, scope=""):
    with tf.variable_scope("feature_ext"+scope, reuse= reuse):
        h = tf.expand_dims(x, axis=-1)
        h = tf.layers.conv1d(h, 10, 3, activation=tf.nn.relu, name="f_conv1", kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
        h = tf.layers.dropout(h, training=training)
        h = tf.layers.conv1d(h, 10, 3, activation=tf.nn.relu, name="f_conv2", kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
        h = tf.layers.dropout(h, training=training)
        h = tf.layers.conv1d(h, 10, 3, activation=tf.nn.relu, name="f_conv3", kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
        h = tf.layers.dropout(h, training=training)
        h = tf.layers.flatten(h, name="f_flat")
        h = tf.layers.dense(h, args.size, activation=tf.nn.relu, name="f_fc", kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
        h = tf.layers.dropout(h, training=training)
    return h


def clf(x, training=True, reuse=False, scope=""):
    with tf.variable_scope("clf"+scope, reuse= reuse):
        h = tf.layers.dense(x, args.size, activation=tf.nn.relu, name="clf_fc1")
        h = tf.layers.dense(h, 4, name="clf_fc", kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
    return h


def discriminator(x, training=True, reuse=False, scope=0):
    with tf.variable_scope("discrimiator"+str(scope), reuse= not training or reuse):
        h = tf.layers.dense(x, args.size*2, activation=tf.nn.relu, name="dis_fc1", kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
        h = tf.layers.dense(h, 2, name="clf_fc")
    return h

def grl(src, tgt, y, training, mode="uni"):
    if training:
        feat_src = feature_extractor(src, training=training)
        logits = clf(feat_src, training=training)
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
        correct_prediction = tf.equal(y, tf.argmax(logits, axis=-1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        global_step = tf.Variable(0., trainable=False, name='global_step')
        p = args.p
        if mode == "dann":
            feat_tgt = feature_extractor(tgt, training=training, reuse=True)
            feat = tf.concat([feat_src, feat_tgt], axis=0)
            feat = flip_gradient(feat, p)
            logits_dm = discriminator(feat)
            labels_dm = tf.concat([tf.ones([args.batch_size], tf.int32), tf.zeros([args.batch_size], tf.int32)], axis=0)
            loss_dm = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_dm, logits=logits_dm))
            loss_total = loss  + loss_dm
        elif mode == "cdan":
            feat_tgt = feature_extractor(tgt, training=training, reuse=True)
            logits_tgt = clf(feat_tgt, training=False, reuse=True)
            softmax_output_tgt = tf.stop_gradient(tf.nn.softmax(logits_tgt, axis=-1))
            softmax_output_src = tf.stop_gradient(tf.nn.softmax(logits, axis=-1))
            feat_src_ = tf.reshape(tf.matmul(tf.reshape(softmax_output_src, [-1, 4, 1]), tf.reshape(feat_src, [-1, 1, args.size])), [-1, 4*args.size])
            feat_tgt_ = tf.reshape(tf.matmul(tf.reshape(softmax_output_tgt, [-1, 4, 1]), tf.reshape(feat_tgt, [-1, 1, args.size])), [-1, 4*args.size])
            feat = tf.concat([feat_src_, feat_tgt_], axis=0)
            feat = flip_gradient(feat, p)
            logits_dm = discriminator(feat)
            labels_dm = tf.concat([tf.ones([args.batch_size], tf.int32), tf.zeros([args.batch_size], tf.int32)], axis=0)
            loss_dm = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_dm, logits=logits_dm))
            loss_total = loss  + loss_dm
        elif mode == "cdanmixup":
            dist_beta = tf.distributions.Beta(args.beta, args.beta)
            lmb = tf.reshape(dist_beta.sample(tf.shape(src)[0]), [-1, 1])
            labels_dm = tf.concat([tf.ones([args.batch_size], tf.int32), tf.zeros([args.batch_size], tf.int32)], axis=0)
            feat_tgt = feature_extractor(tgt, training=training, reuse=True)
            logits_tgt = clf(feat_tgt, training=False, reuse=True)
            softmax_output_tgt = tf.stop_gradient(tf.nn.softmax(logits_tgt, axis=-1))
            softmax_output_src = tf.stop_gradient(tf.nn.softmax(logits, axis=-1))
            idxx = tf.random.shuffle(tf.range(tf.shape(feat_src)[0]))
            feat_src = lmb * feat_src + (1.-lmb) * tf.gather(feat_src, idxx)
            feat_tgt = lmb * feat_tgt + (1.-lmb) * tf.gather(feat_tgt, idxx)
            softmax_output_src = lmb * softmax_output_src + (1.-lmb) * tf.gather(softmax_output_src, idxx)
            softmax_output_tgt = lmb * softmax_output_tgt + (1.-lmb) * tf.gather(softmax_output_tgt, idxx)
            feat_src_ = tf.reshape(tf.matmul(tf.reshape(softmax_output_src, [-1, 4, 1]), tf.reshape(feat_src, [-1, 1, args.size])), [-1, 4*args.size])
            feat_tgt_ = tf.reshape(tf.matmul(tf.reshape(softmax_output_tgt, [-1, 4, 1]), tf.reshape(feat_tgt, [-1, 1, args.size])), [-1, 4*args.size])
            feat = tf.concat([feat_src_, feat_tgt_], axis=0)
            feat = flip_gradient(feat, p)
            logits_dm = discriminator(feat)
            loss_dm = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_dm, logits=logits_dm))
            loss_total = loss  + loss_dm
        else:
            loss_total = loss 

        train_op = tf.train.AdamOptimizer(learning_rate=args.lr).minimize(loss_total, global_step=global_step)
        return loss_total, train_op
    else:
        feat_src = feature_extractor(src, training=False, reuse=True)
        logits = clf(feat_src, training=False, reuse=True)
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
        pred = tf.argmax(logits, axis=-1)
        correct_prediction = tf.equal(y, pred)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return loss, accuracy, pred, y
        

def run():
    tf.keras.backend.clear_session()
    tf.set_random_seed(args.seed)
    seed(args.seed)
    data_src = data_src_fft.astype(np.float32)
    data_tgt = data_tgt_fft.astype(np.float32)
    data_tgt_up = data_tgt_fft_up.astype(np.float32)
    
    args.length = len(data_src)

    dataset_src = tf.data.Dataset.from_tensor_slices((data_src, label_src)).repeat(args.num_epoch).shuffle(len(label_src)).batch(args.batch_size).prefetch(10)
    iterator_src = dataset_src.make_initializable_iterator()
    src_x, src_y = iterator_src.get_next()

    data_val, label_val = data_tgt[:], label_tgt[:]
    
    dataset_tgt = tf.data.Dataset.from_tensor_slices((data_tgt_up, label_tgt_up)).repeat(args.num_epoch * (len(data_src) // len(data_tgt_up) + 1)).shuffle(len(label_tgt_up)).batch(args.batch_size).prefetch(10)
    dataset_val = tf.data.Dataset.from_tensors((data_val, label_val))
    iterator_tgt = dataset_tgt.make_initializable_iterator()
    iterator_val = dataset_val.make_initializable_iterator()

    train_init_op = [iterator_src.initializer, iterator_tgt.initializer]
    test_init_op = iterator_val.initializer
    tgt_x, tgt_y = iterator_tgt.get_next()
    test_x, test_y = iterator_val.get_next()
    
    train_loss, train_op = grl(src_x, tgt_x, src_y, training=True, mode=args.model)
    test_loss, acc_tgt, pred, ground_truth = grl(test_x, test_x, test_y, training=False)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run([train_init_op])
        for ep in range(args.num_epoch):
            for j in range(len(data_src)//args.batch_size):
                train_loss_, _= sess.run([train_loss, train_op])
            sess.run([test_init_op])
            test_loss_, acc_, pred_, ground_truth_ = sess.run([test_loss, acc_tgt, pred, ground_truth])
        print("Test Accuracy" ,acc_)
        balanced_acc = eval(pred_, ground_truth_)
        print("Test Balanced Accuracy", balanced_acc)
    return balanced_acc, acc_



## Source only baseline

In [ ]:
data_src_fft_ori = np.load("XsynallDEenv.npy")
label_src_ori = np.load("ysynallDEenv.npy")
data_tgt_fft_ori = np.load("XreallDEenv.npy")
label_tgt_ori = np.load("yreallDEenv.npy")
# group inner race/outer race/ball faults together
label_src_ori = np.ceil(label_src_ori/3.).astype(np.int64) 
label_tgt_ori = np.ceil(label_tgt_ori/3.).astype(np.int64)

# source data
idx0 = np.where(label_src_ori==0)
idx1 = np.where(label_src_ori==1)
idx2 = np.where(label_src_ori==2)
idx3 = np.where(label_src_ori==3)

# avoid healthy sample overlap
seed(0)
idx_healthy = idx0[0].copy()
np.random.shuffle(idx_healthy)
num_healthy = len(idx_healthy)
idx_src_healthy = np.concatenate([idx_healthy[:num_healthy//2]]*6, axis=0)
idx_tgt_healthy = np.concatenate([idx_healthy[num_healthy//2:]]*6, axis=0)
# balance source classes
idx = np.concatenate([idx_src_healthy, idx1[0], idx2[0], idx3[0]], axis=0)
label_src = label_src_ori[idx]
data_src_fft = data_src_fft_ori[idx]

# Target test data, avoid healthy sample overlap
data_tgt_fft = np.concatenate([data_tgt_fft_ori[label_tgt_ori!=0], data_src_fft_ori[idx_tgt_healthy]], axis=0)
label_tgt = np.concatenate([label_tgt_ori[label_tgt_ori!=0], label_src_ori[idx_tgt_healthy]], axis=0)

# Target training data not used 
label_tgt_up = np.zeros_like(label_tgt.copy())
data_tgt_fft_up = data_tgt_fft.copy()

print("source", "0", np.sum(label_src==0),"1",np.sum(label_src==1), "2",np.sum(label_src==2), "3", np.sum(label_src==3),)
print("target train", "0", np.sum(label_tgt_up==0),"1",np.sum(label_tgt_up==1), "2",np.sum(label_tgt_up==2), "3", np.sum(label_tgt_up==3),)
print("target test", "0", np.sum(label_tgt==0),"1",np.sum(label_tgt==1), "2",np.sum(label_tgt==2), "3", np.sum(label_tgt==3),)

class args:
    batch_size=128
    size=256
    num_epoch=100
    lr=0.001
    model="baseline"
    p=0.1 # meta info not used by baseline

res = []
res_im = []

for i in range(10): 
    args.seed=i
    acc_b, acc_i = run()
    res.append(acc_b)
    res_im.append(acc_i)

print("balanced acuracy", np.mean(res), "+-" , np.std(res))
print("acuracy",np.mean(res_im), "+-" , np.std(res_im))

source 0 1200 1 1200 2 1200 3 1200
target train 0 4800 1 0 2 0 3 0
target test 0 1200 1 1200 2 1200 3 1200
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers

## Balanced DANN - shows the upper-limit for DANN when all classes are balanced


In [ ]:
data_src_fft_ori = np.load("XsynallDEenv.npy")
label_src_ori = np.load("ysynallDEenv.npy")
data_tgt_fft_ori = np.load("XreallDEenv.npy")
label_tgt_ori = np.load("yreallDEenv.npy")
# group inner race/outer race/ball faults together
label_src_ori = np.ceil(label_src_ori/3.).astype(np.int64) 
label_tgt_ori = np.ceil(label_tgt_ori/3.).astype(np.int64)

# source data
idx0 = np.where(label_src_ori==0)
idx1 = np.where(label_src_ori==1)
idx2 = np.where(label_src_ori==2)
idx3 = np.where(label_src_ori==3)

# avoid healthy sample overlap
seed(0)
idx_healthy = idx0[0].copy()
np.random.shuffle(idx_healthy)
num_healthy = len(idx_healthy)
idx_src_healthy = np.concatenate([idx_healthy[:num_healthy//2]]*6, axis=0)
idx_tgt_healthy = np.concatenate([idx_healthy[num_healthy//2:]]*6, axis=0)
# balance source classes
idx = np.concatenate([idx_src_healthy, idx1[0], idx2[0], idx3[0]], axis=0)
label_src = label_src_ori[idx]
data_src_fft = data_src_fft_ori[idx]

# Target test data, avoid healthy sample overlap
data_tgt_fft = np.concatenate([data_tgt_fft_ori[label_tgt_ori!=0], data_src_fft_ori[idx_tgt_healthy]], axis=0)
label_tgt = np.concatenate([label_tgt_ori[label_tgt_ori!=0], label_src_ori[idx_tgt_healthy]], axis=0)

# Target training data not used 
label_tgt_up = label_tgt.copy()
data_tgt_fft_up = data_tgt_fft.copy()

print("source", "0", np.sum(label_src==0),"1",np.sum(label_src==1), "2",np.sum(label_src==2), "3", np.sum(label_src==3),)
print("target train", "0", np.sum(label_tgt_up==0),"1",np.sum(label_tgt_up==1), "2",np.sum(label_tgt_up==2), "3", np.sum(label_tgt_up==3),)
print("target test", "0", np.sum(label_tgt==0),"1",np.sum(label_tgt==1), "2",np.sum(label_tgt==2), "3", np.sum(label_tgt==3),)

label_tgt_up = np.zeros_like(label_tgt_up)

res = []
res_im = []
class args:
    batch_size=128
    size=256
    num_epoch=100
    lr=0.001
    model="dann"
    p=0.1

for i in range(10): 
    args.seed=i 
    acc_b, acc_i = run()
    res.append(acc_b)
    res_im.append(acc_i)
print("balanced acuracy", np.mean(res), "+-" , np.std(res))
print("acuracy", np.mean(res_im), "+-" , np.std(res_im))

source 0 1200 1 1200 2 1200 3 1200
target train 0 1200 1 1200 2 1200 3 1200
target test 0 1200 1 1200 2 1200 3 1200
Test Accuracy 0.84270835
per class 1.0 1.0 0.7041666666666667 0.6666666666666666
Test Balanced Accuracy 0.8427083333333333
Test Accuracy 0.83229166
per class 0.98 1.0 0.6825 0.6666666666666666
Test Balanced Accuracy 0.8322916666666667
Test Accuracy 0.79833335
per class 1.0 1.0 0.5266666666666666 0.6666666666666666
Test Balanced Accuracy 0.7983333333333332
Test Accuracy 0.7914583
per class 1.0 1.0 0.49833333333333335 0.6675
Test Balanced Accuracy 0.7914583333333334
Test Accuracy 0.78541666
per class 1.0 1.0 0.475 0.6666666666666666
Test Balanced Accuracy 0.7854166666666667
Test Accuracy 0.83229166
per class 1.0 1.0 0.6625 0.6666666666666666
Test Balanced Accuracy 0.8322916666666667
Test Accuracy 0.84583336
per class 1.0 1.0 0.7166666666666667 0.6666666666666666
Test Balanced Accuracy 0.8458333333333333
Test Accuracy 0.7972917
per class 1.0 1.0 0.5225 0.6666666666666666
Tes

In [ ]:
data_src_fft_ori = np.load("XsynallDEenv.npy")
label_src_ori = np.load("ysynallDEenv.npy")
data_tgt_fft_ori = np.load("XreallDEenv.npy")
label_tgt_ori = np.load("yreallDEenv.npy")
# group inner race/outer race/ball faults together
label_src_ori = np.ceil(label_src_ori/3.).astype(np.int64) 
label_tgt_ori = np.ceil(label_tgt_ori/3.).astype(np.int64)

# source data
idx0 = np.where(label_src_ori==0)
idx1 = np.where(label_src_ori==1)
idx2 = np.where(label_src_ori==2)
idx3 = np.where(label_src_ori==3)

# avoid healthy sample overlap
seed(0)
idx_healthy = idx0[0].copy()
np.random.shuffle(idx_healthy)
num_healthy = len(idx_healthy)
idx_src_healthy = np.concatenate([idx_healthy[:num_healthy//2]]*6, axis=0)
idx_tgt_healthy = np.concatenate([idx_healthy[num_healthy//2:]]*6, axis=0)
# balance source classes
idx = np.concatenate([idx_src_healthy, idx1[0], idx2[0], idx3[0]], axis=0)
label_src = label_src_ori[idx]
data_src_fft = data_src_fft_ori[idx]

# Target test data, avoid healthy sample overlap
data_tgt_fft = np.concatenate([data_tgt_fft_ori[label_tgt_ori!=0], data_src_fft_ori[idx_tgt_healthy]], axis=0)
label_tgt = np.concatenate([label_tgt_ori[label_tgt_ori!=0], label_src_ori[idx_tgt_healthy]], axis=0)


# Target training data
idx0 = np.where(label_tgt==0)
idx1 = np.where(label_tgt==1)
idx2 = np.where(label_tgt==2)
idx3 = np.where(label_tgt==3)
idx00 = idx0[0] # Healthy 1200 100%
idx11 = np.random.choice(idx1[0], int(len(idx1[0]) * 0.05)) # Inner 10%
idx22 = np.random.choice(idx2[0], int(len(idx2[0]) * 0.01)) # Rolling 1%
idx33 = np.random.choice(idx3[0], int(len(idx3[0]) * 0.1)) # Outer 20%
idx = np.concatenate([idx00 , idx11 , idx22 , idx33], axis=0)
label_tgt_up = label_tgt[idx]
data_tgt_fft_up = data_tgt_fft[idx]


print("source", "0", np.sum(label_src==0),"1",np.sum(label_src==1), "2",np.sum(label_src==2), "3", np.sum(label_src==3),)
print("target train", "0", np.sum(label_tgt_up==0),"1",np.sum(label_tgt_up==1), "2",np.sum(label_tgt_up==2), "3", np.sum(label_tgt_up==3),)
print("target test", "0", np.sum(label_tgt==0),"1",np.sum(label_tgt==1), "2",np.sum(label_tgt==2), "3", np.sum(label_tgt==3),)


label_tgt_up = np.zeros_like(label_tgt_up) #mask out target labels

class args:
    batch_size=128
    size=256
    num_epoch=100
    lr=0.001
    model="dann"
    p=0.1


res = []
res_im = []
for i in range(10): 
    args.seed=i    
    acc_b, acc_i = run()
    res.append(acc_b)
    res_im.append(acc_i)
print("balanced acuracy", np.mean(res), "+-" , np.std(res))
print("acuracy", np.mean(res_im), "+-" , np.std(res_im))

source 0 1200 1 1200 2 1200 3 1200
target train 0 1200 1 60 2 12 3 120
target test 0 1200 1 1200 2 1200 3 1200
Test Accuracy 0.72541666
per class 0.995 1.0 0.17583333333333334 0.7308333333333333
Test Balanced Accuracy 0.7254166666666667
Test Accuracy 0.69354165
per class 1.0 1.0 0.10166666666666667 0.6725
Test Balanced Accuracy 0.6935416666666666
Test Accuracy 0.76666665
per class 1.0 1.0 0.315 0.7516666666666667
Test Balanced Accuracy 0.7666666666666666
Test Accuracy 0.70145833
per class 0.995 1.0 0.07666666666666666 0.7341666666666666
Test Balanced Accuracy 0.7014583333333333
Test Accuracy 0.68104166
per class 1.0 1.0 0.006666666666666667 0.7175
Test Balanced Accuracy 0.6810416666666668
Test Accuracy 0.6897917
per class 1.0 1.0 0.030833333333333334 0.7283333333333334
Test Balanced Accuracy 0.6897916666666667
Test Accuracy 0.6983333
per class 1.0 1.0 0.005833333333333334 0.7875
Test Balanced Accuracy 0.6983333333333334
Test Accuracy 0.7783333
per class 1.0 1.0 0.44416666666666665 0.66

## CDAN - Improved over DANN

In [ ]:
data_src_fft_ori = np.load("XsynallDEenv.npy")
label_src_ori = np.load("ysynallDEenv.npy")
data_tgt_fft_ori = np.load("XreallDEenv.npy")
label_tgt_ori = np.load("yreallDEenv.npy")
# group inner race/outer race/ball faults together
label_src_ori = np.ceil(label_src_ori/3.).astype(np.int64) 
label_tgt_ori = np.ceil(label_tgt_ori/3.).astype(np.int64)

# source data
idx0 = np.where(label_src_ori==0)
idx1 = np.where(label_src_ori==1)
idx2 = np.where(label_src_ori==2)
idx3 = np.where(label_src_ori==3)

# avoid healthy sample overlap
seed(0)
idx_healthy = idx0[0].copy()
np.random.shuffle(idx_healthy)
num_healthy = len(idx_healthy)
idx_src_healthy = np.concatenate([idx_healthy[:num_healthy//2]]*6, axis=0)
idx_tgt_healthy = np.concatenate([idx_healthy[num_healthy//2:]]*6, axis=0)
# balance source classes
idx = np.concatenate([idx_src_healthy, idx1[0], idx2[0], idx3[0]], axis=0)
label_src = label_src_ori[idx]
data_src_fft = data_src_fft_ori[idx]

# Target test data, avoid healthy sample overlap
data_tgt_fft = np.concatenate([data_tgt_fft_ori[label_tgt_ori!=0], data_src_fft_ori[idx_tgt_healthy]], axis=0)
label_tgt = np.concatenate([label_tgt_ori[label_tgt_ori!=0], label_src_ori[idx_tgt_healthy]], axis=0)


# Target training data
idx0 = np.where(label_tgt==0)
idx1 = np.where(label_tgt==1)
idx2 = np.where(label_tgt==2)
idx3 = np.where(label_tgt==3)
idx00 = idx0[0] # Healthy 1200 100%
idx11 = np.random.choice(idx1[0], int(len(idx1[0]) * 0.05)) # Inner 10%
idx22 = np.random.choice(idx2[0], int(len(idx2[0]) * 0.01)) # Rolling 1%
idx33 = np.random.choice(idx3[0], int(len(idx3[0]) * 0.1)) # Outer 20%
idx = np.concatenate([idx00 , idx11 , idx22 , idx33], axis=0)
label_tgt_up = label_tgt[idx]
data_tgt_fft_up = data_tgt_fft[idx]

print("source", "0", np.sum(label_src==0),"1",np.sum(label_src==1), "2",np.sum(label_src==2), "3", np.sum(label_src==3),)
print("target train", "0", np.sum(label_tgt_up==0),"1",np.sum(label_tgt_up==1), "2",np.sum(label_tgt_up==2), "3", np.sum(label_tgt_up==3),)
print("target test", "0", np.sum(label_tgt==0),"1",np.sum(label_tgt==1), "2",np.sum(label_tgt==2), "3", np.sum(label_tgt==3),)


label_tgt_up = np.zeros_like(label_tgt_up) #mask out target labels

class args:
    batch_size=128
    size=256
    num_epoch=100
    lr=0.001
    model="cdan"
    p=0.1

res = []
res_im = []

for i in range(10): 
    args.seed=i   
    acc_b, acc_i = run()
    res.append(acc_b)
    res_im.append(acc_i)
print("balanced acuracy", np.mean(res), "+-" , np.std(res))
print("acuracy", np.mean(res_im), "+-" , np.std(res_im))

source 0 1200 1 1200 2 1200 3 1200
target train 0 1200 1 60 2 12 3 120
target test 0 1200 1 1200 2 1200 3 1200
Test Accuracy 0.73270833
per class 1.0 1.0 0.2625 0.6683333333333333
Test Balanced Accuracy 0.7327083333333334
Test Accuracy 0.69854164
per class 1.0 1.0 0.1275 0.6666666666666666
Test Balanced Accuracy 0.6985416666666666
Test Accuracy 0.768125
per class 1.0 1.0 0.3875 0.685
Test Balanced Accuracy 0.7681250000000001
Test Accuracy 0.75604165
per class 0.995 1.0 0.3566666666666667 0.6725
Test Balanced Accuracy 0.7560416666666666
Test Accuracy 0.76104164
per class 1.0 1.0 0.3775 0.6666666666666666
Test Balanced Accuracy 0.7610416666666666
Test Accuracy 0.763125
per class 1.0 1.0 0.3775 0.675
Test Balanced Accuracy 0.763125
Test Accuracy 0.75125
per class 1.0 1.0 0.3375 0.6675
Test Balanced Accuracy 0.75125
Test Accuracy 0.7825
per class 1.0 1.0 0.44916666666666666 0.6808333333333333
Test Balanced Accuracy 0.7825
Test Accuracy 0.765625
per class 1.0 1.0 0.3958333333333333 0.666666

## Proposed conditional + Mixup

In [ ]:
data_src_fft_ori = np.load("XsynallDEenv.npy")
label_src_ori = np.load("ysynallDEenv.npy")
data_tgt_fft_ori = np.load("XreallDEenv.npy")
label_tgt_ori = np.load("yreallDEenv.npy")
# group inner race/outer race/ball faults together
label_src_ori = np.ceil(label_src_ori/3.).astype(np.int64) 
label_tgt_ori = np.ceil(label_tgt_ori/3.).astype(np.int64)

# source data
idx0 = np.where(label_src_ori==0)
idx1 = np.where(label_src_ori==1)
idx2 = np.where(label_src_ori==2)
idx3 = np.where(label_src_ori==3)

# avoid healthy sample overlap
seed(0)
idx_healthy = idx0[0].copy()
np.random.shuffle(idx_healthy)
num_healthy = len(idx_healthy)
idx_src_healthy = np.concatenate([idx_healthy[:num_healthy//2]]*6, axis=0)
idx_tgt_healthy = np.concatenate([idx_healthy[num_healthy//2:]]*6, axis=0)
# balance source classes
idx = np.concatenate([idx_src_healthy, idx1[0], idx2[0], idx3[0]], axis=0)
label_src = label_src_ori[idx]
data_src_fft = data_src_fft_ori[idx]

# Target test data, avoid healthy sample overlap
data_tgt_fft = np.concatenate([data_tgt_fft_ori[label_tgt_ori!=0], data_src_fft_ori[idx_tgt_healthy]], axis=0)
label_tgt = np.concatenate([label_tgt_ori[label_tgt_ori!=0], label_src_ori[idx_tgt_healthy]], axis=0)


# Target training data
idx0 = np.where(label_tgt==0)
idx1 = np.where(label_tgt==1)
idx2 = np.where(label_tgt==2)
idx3 = np.where(label_tgt==3)
idx00 = idx0[0] # Healthy 1200 100%
idx11 = np.random.choice(idx1[0], int(len(idx1[0]) * 0.05)) # Inner 10%
idx22 = np.random.choice(idx2[0], int(len(idx2[0]) * 0.01)) # Rolling 1%
idx33 = np.random.choice(idx3[0], int(len(idx3[0]) * 0.1)) # Outer 20%
idx = np.concatenate([idx00 , idx11 , idx22 , idx33], axis=0)
label_tgt_up = label_tgt[idx]
data_tgt_fft_up = data_tgt_fft[idx]

print("source", "0", np.sum(label_src==0),"1",np.sum(label_src==1), "2",np.sum(label_src==2), "3", np.sum(label_src==3),)
print("target train", "0", np.sum(label_tgt_up==0),"1",np.sum(label_tgt_up==1), "2",np.sum(label_tgt_up==2), "3", np.sum(label_tgt_up==3),)
print("target test", "0", np.sum(label_tgt==0),"1",np.sum(label_tgt==1), "2",np.sum(label_tgt==2), "3", np.sum(label_tgt==3),)

label_tgt_up = np.zeros_like(label_tgt_up) #mask out target labels

res = []
res_im = []
class args:
    batch_size=128
    size=256
    num_epoch=100
    lr=0.001
    model="cdanmixup"
    p=0.1
    beta=1.

for i in range(10): 
    args.seed=i  
    acc_b, acc_i = run()
    res.append(acc_b)
    res_im.append(acc_i)
print("balanced acuracy", np.mean(res), "+-" , np.std(res))
print("acuracy", np.mean(res_im), "+-" , np.std(res_im))

source 0 1200 1 1200 2 1200 3 1200
target train 0 1200 1 60 2 12 3 120
target test 0 1200 1 1200 2 1200 3 1200
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Test Accuracy 0.82625
per class 1.0 1.0 0.6383333333333333 0.6666666666666666
Test Balanced Accuracy 0.8262499999999999


/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Test Accuracy 0.84166664
per class 1.0 1.0 0.7 0.6666666666666666
Test Balanced Accuracy 0.8416666666666667


...

Test Accuracy 0.825
per class 1.0 1.0 0.6333333333333333 0.6666666666666666
Test Balanced Accuracy 0.825


/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Test Accuracy 0.81854165
per class 1.0 1.0 0.6075 0.6666666666666666
Test Balanced Accuracy 0.8185416666666666


/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Test Accuracy 0.83270836
per class 1.0 1.0 0.6641666666666667 0.6666666666666666
Test Balanced Accuracy 0.8327083333333333
balanced acuracy 0.8228958333333333 +- 0.021383252549210852
acuracy 0.8228958 +- 0.021383256
